In [1]:
#ADIBATIC EXPANSION
import pygame
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt

dt = 1

# Set up the ball properties
radius = 5
num_balls = 400

# Set up the box properties
Lxn = -200
Lxp = 200
Lyn = -200
Lyp = 200

x = np.random.uniform(Lxn+radius, Lxp-radius, num_balls)
y = np.random.uniform(Lyn+radius, Lyp-radius, num_balls)
positions = np.column_stack((x, y))

vx = np.random.randn(num_balls)
vy = np.random.randn(num_balls)
velocities = np.column_stack((vx, vy))

# Set up the screen
width, height = 1600, 800
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Bouncing Balls")

# Set up the colors
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)
black = (0, 0, 0)
white = (255, 255, 255)

#newlines
# Parameters for equilibration phase
equilibrium_steps = 1000  # Adjust as needed
equilibrium_temperature_threshold = 0.01  # Adjust as needed

# Equilibration phase
equilibrating = True
equilibrium_steps_count = 0
equilibrium_temperatures = []

while equilibrating:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            equilibrating = False

    # Calculate temperature during equilibration
    temperature = 0.5 * np.sum(np.linalg.norm(velocities, axis=1) ** 2) / num_balls
    equilibrium_temperatures.append(temperature)

    # Check for equilibrium based on temperature threshold
    if len(equilibrium_temperatures) > equilibrium_steps:
        temperature_change = np.max(np.abs(np.diff(equilibrium_temperatures[-equilibrium_steps:])))
        if temperature_change < equilibrium_temperature_threshold:
            equilibrating = False
            
# Game loop
running = True
clock = pygame.time.Clock()

vLxp = 0

M = 10
Q = 50
x_min = Lxn
y_min = Lyn
x_max = 600
y_max = Lyp
v_min = -5
v_max = 5
states = np.zeros((Q,M,M,M,M))

#newlines
# Reset parameters for adiabatic expansion
Lxp_original = Lxp  # Save the original Lxp value
Lxp += vLxp * dt  # Move the wall outward



t = 0
entropy = []
temperature = []
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    Lxp += vLxp*dt
    
    if t > 200:
        vLxp = 0.02

    q = t % Q

    states[q] = 0
    for n in range(num_balls):
        i = int((positions[n,0]-x_min)/(x_max-x_min)*M)
        j = int((positions[n,1]-y_min)/(y_max-y_min)*M)
        k = int((velocities[n,0]-v_min)/(v_max-v_min)*M)
        l = int((velocities[n,1]-v_min)/(v_max-v_min)*M)
        states[q,i,j,k,l] += 1

    # Calculate entropy
    p = np.sum(states, axis=0).flatten()
    p = p/np.sum(p)

    nz = np.nonzero(p)
    entropy.append(-np.sum(p[nz]*np.log(p[nz])))

    # Calculate the temperature
    temperature.append(0.5*np.sum(np.linalg.norm(velocities, axis=1)**2)/num_balls)


    # Sort with respect to x
    sort_index = np.argsort(positions[:,0])
    positions = positions[sort_index]
    velocities = velocities[sort_index]

    # Update ball positions
    positions += velocities*dt

    # Check for collisions with walls
    for i in range(num_balls):
        if positions[i][0] < Lxn + radius and velocities[i][0] < 0:
            velocities[i][0] = -velocities[i][0]
        if positions[i][1] < Lyn + radius and velocities[i][1] < 0:
            velocities[i][1] = -velocities[i][1]
        if positions[i][1] > Lyp - radius and velocities[i][1] > 0:
            velocities[i][1] = -velocities[i][1]

        # adiabatic expansion
        if positions[i][0] > Lxp - radius and velocities[i][0] > vLxp:
            velocities[i][0] = -velocities[i][0] + 2*vLxp

    # Check for collisions between balls
    for i in range(num_balls):
        for j in range(i+1, num_balls):
            if (positions[j,0] - positions[i,0] > 2*radius):
                break
            else:
                rp = positions[j] - positions[i]
                nrp = np.linalg.norm(rp)
                if nrp < 2*radius:
                    rv = velocities[j] - velocities[i]
                    if np.dot(rv,rp) < 0:
                        vp = np.dot(rv,rp)*rp/nrp**2
                        velocities[i] += vp
                        velocities[j] -= vp

    t += 1
            
    # Clear the screen
    screen.fill(black)

    # Draw the balls
    for i in range(num_balls):
        pygame.draw.circle(screen, white, \
        (int(positions[i][0]+300), int(positions[i][1]+400)), radius)
    
    # Draw the box
    pygame.draw.rect(screen, green, \
    (300 + Lxn, 400 + Lyn, Lxp - Lxn, Lyp - Lyn), 2)

    # Draw entropy
    for i in range(t):
        pygame.draw.circle(screen, red, \
        (1000 + i/10, 600 - int((entropy[i]-entropy[0])*300)), 2)

    # Draw temperature
    for i in range(t):
        pygame.draw.circle(screen, green, \
        (1000 + i/10, 100 - int((temperature[i]-temperature[0])*300)), 2)

    # Update the display
    pygame.display.flip()
    clock.tick(30)

# Quit the game
pygame.quit()


pygame 2.5.2 (SDL 2.28.3, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
#P-V DIAGRAM
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt

dt = 1

# Set up the ball properties
radius = 5
num_balls = 200

# Set up the box properties
Lxn = -200
Lxp = 200
Lyn = -200
Lyp = 200

x = np.random.uniform(Lxn+radius, Lxp-radius, num_balls)
y = np.random.uniform(Lyn+radius, Lyp-radius, num_balls)
positions = np.column_stack((x, y))

vx = np.random.randn(num_balls)
vy = np.random.randn(num_balls)
velocities = np.column_stack((vx, vy))

v_wall = 0.01

M = 10
Q = 1000
N_cycle = 20000
x_min = Lxn
y_min = Lyn
x_max = Lxp + v_wall*N_cycle/2*dt
y_max = Lyp
v_min = -6
v_max = 6
states = np.zeros((Q,M,M,M,M))
q_pressures = np.zeros((Q,1))

for t in range(1000):
    # Sort with respect to x
    sort_index = np.argsort(positions[:,0])
    positions = positions[sort_index]
    velocities = velocities[sort_index]

    q = t % Q

    states[q] = 0
    for n in range(num_balls):
        i = int((positions[n,0]-x_min)/(x_max-x_min)*M)
        j = int((positions[n,1]-y_min)/(y_max-y_min)*M)
        k = int((velocities[n,0]-v_min)/(v_max-v_min)*M)
        l = int((velocities[n,1]-v_min)/(v_max-v_min)*M)
        states[q,i,j,k,l] += 1

    # Update ball positions
    positions += velocities*dt

    q_pressures[q] = 0

    # Check for collisions with walls
    for i in range(num_balls):
        if positions[i][0] < Lxn + radius and velocities[i][0] < 0:
            velocities[i][0] = -velocities[i][0]
            q_pressures[q] += 2*velocities[i][0]/(Lyp - Lyn)
        if positions[i][1] < Lyn + radius and velocities[i][1] < 0:
            velocities[i][1] = -velocities[i][1]
        if positions[i][1] > Lyp - radius and velocities[i][1] > 0:
            velocities[i][1] = -velocities[i][1]
        if positions[i][0] > Lxp - radius and velocities[i][0] > 0:
            velocities[i][0] = -velocities[i][0]

    # Check for collisions between balls
    for i in range(num_balls):
        for j in range(i+1, num_balls):
            if (positions[j,0] - positions[i,0] > 2*radius):
                break
            else:
                rp = positions[j] - positions[i]
                nrp = np.linalg.norm(rp)
                if nrp < 2*radius:
                    rv = velocities[j] - velocities[i]
                    if np.dot(rv,rp) < 0:
                        vp = np.dot(rv,rp)*rp/nrp**2
                        velocities[i] += vp
                        velocities[j] -= vp

print('Thermalization complete')


entropy = []
temperature = []
pressure = []
volume = []

for t in range(N_cycle):
    
    stage = int((t % N_cycle)/N_cycle*4)
    volume.append((Lxp - Lxn)*(Lyp-Lyn))

    if stage == 0 or stage == 1:
        vLxp = v_wall
    else:
        vLxp = -v_wall

    Lxp += vLxp*dt

    q = t % Q

    states[q] = 0
    for n in range(num_balls):
        i = int((positions[n,0]-x_min)/(x_max-x_min)*M)
        j = int((positions[n,1]-y_min)/(y_max-y_min)*M)
        k = int((velocities[n,0]-v_min)/(v_max-v_min)*M)
        l = int((velocities[n,1]-v_min)/(v_max-v_min)*M)
        states[q,i,j,k,l] += 1

    # Calculate entropy
    p = np.sum(states, axis=0).flatten()
    p = p/np.sum(p)

    nz = np.nonzero(p)
    entropy.append(-np.sum(p[nz]*np.log(p[nz])))

    # Calculate the temperature
    temperature.append(0.5*np.sum(np.linalg.norm(velocities, axis=1)**2)/num_balls)

    # Sort with respect to x
    sort_index = np.argsort(positions[:,0])
    positions = positions[sort_index]
    velocities = velocities[sort_index]

    # Update ball positions
    positions += velocities*dt

    # Calculate pressure
    q_pressures[q] = 0

    # Check for collisions with walls
    for i in range(num_balls):
        if positions[i][0] < Lxn + radius and velocities[i][0] < 0:
            velocities[i][0] = -velocities[i][0]
            q_pressures[q] += 2*velocities[i][0]/(Lyp - Lyn)
        if positions[i][1] < Lyn + radius and velocities[i][1] < 0:
            velocities[i][1] = -velocities[i][1]
        if positions[i][1] > Lyp - radius and velocities[i][1] > 0:
            velocities[i][1] = -velocities[i][1]

        if stage == 0 or stage == 2:
            # isothermal expansion (or compression)
            if positions[i][0] > Lxp - radius and velocities[i][0] > 0:
                velocities[i][0] = -velocities[i][0]
        else:
            # adiabatic expansion (or compression)
            if positions[i][0] > Lxp - radius and velocities[i][0] > vLxp:
                velocities[i][0] = -velocities[i][0] + 2*vLxp

    pressure.append(np.sum(q_pressures, axis=0))

    # Check for collisions between balls
    for i in range(num_balls):
        for j in range(i+1, num_balls):
            if (positions[j,0] - positions[i,0] > 2*radius):
                break
            else:
                rp = positions[j] - positions[i]
                nrp = np.linalg.norm(rp)
                if nrp < 2*radius:
                    rv = velocities[j] - velocities[i]
                    if np.dot(rv,rp) < 0:
                        vp = np.dot(rv,rp)*rp/nrp**2
                        velocities[i] += vp
                        velocities[j] -= vp

plt.plot(entropy[-N_cycle:],temperature[-N_cycle:])
plt.xlabel('Entropy (S)')
plt.ylabel('Temperature (T)')
plt.show()

plt.plot(volume[-N_cycle:],pressure[-N_cycle:])
plt.xlabel('Volume (V)')
plt.ylabel('Pressure (P)')
plt.show()